In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
input_dir= "/kaggle/input/march-machine-learning-mania-2024/"
#pd.read_csv(input_dir+"Conferences.csv")

### possible features

In [ ]:
#point per possession 
#PPP = Pts/Poss 
#POSS = FGA + 0.475 x FTA - ORB + TO
 
# Ast/To ratio

#Effective field goal percentage
# (FGM + 0.5 * 3PM) / FGA.

#True shooting percentage
#PTS/(2*(FGA+0.44*FTA))

#TOV percentage
# 100 * TOV / (FGA + 0.44 * FTA + TOV).

#pace 
# Minutes per Game x ((Team Possessions + Opponent Possessions) ÷ (2 x (Team Minutes Played ÷ 5)))

# MNCAA start from here
## MNCAA Regular Season ↓

In [ ]:
# MNCAA start from here
M_RegularDetail= pd.read_csv(input_dir+"MRegularSeasonDetailedResults.csv")
display(M_RegularDetail)

In [ ]:
#MTeamConferences
MTeamConferences = pd.read_csv(input_dir+"MTeamConferences.csv")
MTeamConferences

In [ ]:
# merge M_RegularDetail with MTeamConferences
MTeamConferences = pd.read_csv(input_dir+"MTeamConferences.csv")
MTeamConferences = MTeamConferences.rename(columns={'TeamID':'WTeamID'})
M_RegularDetail_merge_MTeamConferences = pd.merge(left=M_RegularDetail,right=\
                                             MTeamConferences,on=['Season','WTeamID'],\
                                                  how='left')
M_RegularDetail_merge_MTeamConferences

In [ ]:
#confirm column'ConfAbbrev' without nan
M_RegularDetail_merge_MTeamConferences.dropna(subset='ConfAbbrev')

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
M_WinTeams= pd.DataFrame()
M_LoseTeams=pd.DataFrame()

columns= ['Season','TeamID', 'Score', 'OppScore',
       'Loc', 'NumOT', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA',
       'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'OppFGM', 'OppFGA',
       'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppOR', 'OppDR', 'OppAst', 'OppTO',
       'OppStl', 'OppBlk', 'Opp PF']

M_WinTeams[columns]= M_RegularDetail[['Season','WTeamID', 'WScore',  'LScore',
       'WLoc', 'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA',
       'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA',
       'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO',
       'LStl', 'LBlk', 'LPF']]

M_WinTeams["Win"]=1
M_WinTeams["Loss"]=0

M_LoseTeams[columns]= M_RegularDetail[['Season','LTeamID', 'LScore',  'WScore',
       'WLoc', 'NumOT', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA',
       'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 'WFGM', 'WFGA',
       'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO',
       'WStl', 'WBlk', 'WPF']]

def change_loc(loc):
    if loc == 'H':
        return 'A'
    elif loc == 'A':
        return 'H'
    else:
        return 'N'
    
M_LoseTeams["Loc"]= M_LoseTeams["Loc"].apply(change_loc)
    
M_LoseTeams["Win"]=0
M_LoseTeams ["Loss"]=1

M_WinLoseTeams= pd.concat([M_WinTeams,M_LoseTeams])

M_WinLoseTeams['H'] = (M_WinLoseTeams['Loc'] == 'H').astype(int)
M_WinLoseTeams['A'] = (M_WinLoseTeams['Loc'] == 'A').astype(int)
M_WinLoseTeams['N'] = (M_WinLoseTeams['Loc'] == 'N').astype(int)
M_WinLoseTeams.drop('Loc', axis=1, inplace=True)

M_combinedTeams= M_WinLoseTeams.groupby(["Season","TeamID"]).sum()
M_combinedTeams["Game_played"]= M_combinedTeams["Win"] + M_combinedTeams["Loss"]

display(M_combinedTeams.columns.values)

In [ ]:
M_RegularSeason_input= pd.DataFrame()

M_RegularSeason_input["win_ratio"]= M_combinedTeams["Win"]/M_combinedTeams["Game_played"]
M_RegularSeason_input["ppg"]= M_combinedTeams["Score"]/M_combinedTeams["Game_played"]
M_RegularSeason_input["opp_ppg"]= M_combinedTeams["OppScore"]/M_combinedTeams["Game_played"]
M_RegularSeason_input["point_diff_per_game"]= (M_combinedTeams["Score"]-M_combinedTeams["OppScore"])/M_combinedTeams["Game_played"]
M_RegularSeason_input["ot_per_game"]= M_combinedTeams["NumOT"]/M_combinedTeams["Game_played"]

M_RegularSeason_input["FGM3pergame"]= M_combinedTeams["FGM3"]/M_combinedTeams["Game_played"]
M_RegularSeason_input["FG3ratio"]= M_combinedTeams["FGM3"]/M_combinedTeams["FGA3"]
M_RegularSeason_input["FG3_allowedpergame"]= M_combinedTeams["OppFGM3"]/M_combinedTeams["Game_played"]

M_RegularSeason_input["FGMpergame"]= M_combinedTeams["FGM"]/M_combinedTeams["Game_played"]
M_RegularSeason_input["FGratio"]= M_combinedTeams["FGM"]/M_combinedTeams["FGA"]
M_RegularSeason_input["FG_allowedpergame"]= M_combinedTeams["OppFGM"]/M_combinedTeams["Game_played"]

M_RegularSeason_input["FTMpergame"]= M_combinedTeams["FTM"]/M_combinedTeams["Game_played"]
M_RegularSeason_input["FTratio"]= M_combinedTeams["FTM"]/M_combinedTeams["FTA"]
M_RegularSeason_input["FT_allowedpergame"]= M_combinedTeams["OppFTM"]/M_combinedTeams["Game_played"]


M_RegularSeason_input["ORratio"]= M_combinedTeams["OR"]/(M_combinedTeams["OR"]+M_combinedTeams["OppDR"])
M_RegularSeason_input["DRratio"]= M_combinedTeams["DR"]/(M_combinedTeams["DR"]+M_combinedTeams["OppOR"])

M_RegularSeason_input["Ast_per_g"]= M_combinedTeams["Ast"]/M_combinedTeams["Game_played"]
M_RegularSeason_input["To_per_g"]= M_combinedTeams["TO"]/M_combinedTeams["Game_played"]
M_RegularSeason_input["Stl_per_g"]= M_combinedTeams["Stl"]/M_combinedTeams["Game_played"]
M_RegularSeason_input["Blk_per_g"]= M_combinedTeams["Blk"]/M_combinedTeams["Game_played"]
M_RegularSeason_input["Pf_per_g"]= M_combinedTeams["PF"]/M_combinedTeams["Game_played"]


##

display(M_RegularSeason_input)

In [ ]:
M_seeds = pd.read_csv(input_dir+"MNCAATourneySeeds.csv")
M_TourneyCompact=pd.read_csv(input_dir+"MNCAATourneyCompactResults.csv")


M_seeds_dict= M_seeds.set_index(["Season","TeamID"])
#display(M_seeds_dict.index.values)

M_tourney_input=pd.DataFrame()

M_winIDs= M_TourneyCompact["WTeamID"]
M_loseIDs=M_TourneyCompact["LTeamID"]
M_season= M_TourneyCompact["Season"]

M_winners= pd.DataFrame()
M_winners[["Season","Team1","Team2"]]= M_TourneyCompact[["Season","WTeamID","LTeamID"]]
M_winners["Result"]=1

M_losers= pd.DataFrame()
M_losers[["Season","Team1","Team2"]]= M_TourneyCompact[["Season","LTeamID","WTeamID"]]
M_losers["Result"]=0

M_tourney_input= pd.concat([M_winners,M_losers])

M_tourney_input= M_tourney_input[M_tourney_input["Season"]>2002].reset_index(drop=True)

#M_tourney_input


M_team1_seeds=[]
M_team2_seeds=[]

for i in range(len(M_tourney_input)):
    idx= (M_tourney_input["Season"][i], M_tourney_input["Team1"][i])
    seed= M_seeds_dict.loc[idx].values[0]
    if len(seed)==4:
        seed= int(seed[1:-1])
    else:
        seed= int(seed[1:])
    M_team1_seeds.append(seed)
    
    idx= (M_tourney_input["Season"][i], M_tourney_input["Team2"][i])
    seed= M_seeds_dict.loc[idx].values[0]
    if len(seed)==4:
        seed= int(seed[1:-1])
    else:
        seed= int(seed[1:])
    M_team2_seeds.append(seed)

M_tourney_input["Team1Seed"]=M_team1_seeds
M_tourney_input["Team2Seed"]=M_team2_seeds

M_tourney_input




In [ ]:
M_outscores=[]

for i in range(len(M_tourney_input)):
    idx= (M_tourney_input["Season"][i], M_tourney_input["Team1"][i])
    team1score= M_RegularSeason_input.loc[idx].copy()
    team1score["Seed"]= M_tourney_input["Team1Seed"][i]
    
    idx= (M_tourney_input["Season"][i], M_tourney_input["Team2"][i])
    team2score= M_RegularSeason_input.loc[idx].copy()
    team2score["Seed"]= M_tourney_input["Result"][i]
    
    outscore= team1score-team2score
    outscore["Result"]= M_tourney_input["Team2Seed"][i]
    M_outscores.append(outscore)
    
M_outscores=pd.DataFrame(M_outscores)



In [ ]:
display(M_outscores.describe())
display(M_outscores)

Ray

In [ ]:
#point per possession 
#PPP = Pts/Poss 
#POSS = FGA + 0.475 x FTA - ORB + TO
 
# Ast/To ratio

#Effective field goal percentage
# (FGM + 0.5 * 3PM) / FGA.

#True shooting percentage
#PTS/(2*(FGA+0.44*FTA))

#TOV percentage
# 100 * TOV / (FGA + 0.44 * FTA + TOV).

#pace 
# Minutes per Game x ((Team Possessions + Opponent Possessions) ÷ (2 x (Team Minutes Played ÷ 5)))

In [ ]:
pd.set_option('display.max_columns', None)
# M_RegularDetail_merge_MTeamConferences.head()
M_RegularDetail_merge_MTeamConferences['WPPP']=M_RegularDetail_merge_MTeamConferences['WFGA']+0.475*M_RegularDetail_merge_MTeamConferences['WFTA']
-M_RegularDetail_merge_MTeamConferences['WOR']+M_RegularDetail_merge_MTeamConferences['WTO']
# M_RegularDetail_merge_MTeamConferences['WPPP']

# WNCAA start from here
## WNCAA tourney ↓

In [4]:
input_dir= "/kaggle/input/march-machine-learning-mania-2024/"
reg_det = pd.read_csv(input_dir+"WRegularSeasonDetailedResults.csv")
pd.set_option('display.max_rows', 25)
display(reg_det)
seeds = pd.read_csv(input_dir+"WNCAATourneySeeds.csv")
display(seeds)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/march-machine-learning-mania-2024/WRegularSeasonDetailedResults.csv'

In [ ]:
WTeamConferences = pd.read_csv(input_dir+"WTeamConferences.csv")
WTeamConferences = WTeamConferences.rename(columns={'TeamID':'TeamID'})
W_seeds_merge_WTeamConferences = pd.merge(left=seeds,right=\
                                             WTeamConferences,on=['Season','TeamID'],\
                                                  how='left')

W_seeds_merge_WTeamConferences = pd.get_dummies(W_seeds_merge_WTeamConferences, columns=['ConfAbbrev'])
dummies_columns = [col for col in W_seeds_merge_WTeamConferences if 'ConfAbbrev' in col]

W_seeds_merge_WTeamConferences[dummies_columns] = W_seeds_merge_WTeamConferences[dummies_columns].astype(int)

W_seeds_merge_WTeamConferences

In [ ]:
w_Teams = pd.DataFrame()
l_Teams = pd.DataFrame()

columns = ['Season', 'TeamID', 'Score', 'oppScore',
       'Loc', 'NumOT', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA',
       'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'oppFGM', 'oppFGA',
       'oppFGM3', 'oppFGA3', 'oppFTM', 'oppFTA', 'oppOR', 'oppDR', 'oppAst', 'oppTO',
       'oppStl', 'oppBlk', 'oppPF']

w_Teams[columns] = reg_det[['Season', 'WTeamID', 'WScore', 'LScore',
       'WLoc', 'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA',
       'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA',
       'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO',
       'LStl', 'LBlk', 'LPF']]
w_Teams['win'] = 1
w_Teams['loss'] = 0

columns = ['Season', 'TeamID', 'Score', 'oppScore',
       'Loc', 'NumOT', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA',
       'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'oppFGM', 'oppFGA',
       'oppFGM3', 'oppFGA3', 'oppFTM', 'oppFTA', 'oppOR', 'oppDR', 'oppAst', 'oppTO',
       'oppStl', 'oppBlk', 'oppPF']

l_Teams[columns] = reg_det[['Season', 'LTeamID', 'LScore', 'WScore',
       'WLoc', 'NumOT', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA',
       'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 'WFGM', 'WFGA',
       'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO',
       'WStl', 'WBlk', 'WPF']]
l_Teams['win'] = 0
l_Teams['loss'] = 1

def change_loc(loc):
    if loc == 'H':
        return 'A'
    elif loc == 'A':
        return 'H'
    else:
        return 'N'

l_Teams['Loc'] = l_Teams['Loc'].apply(change_loc)


all_teams = pd.concat([w_Teams, l_Teams])
display(all_teams)
all_teams['H'] = (all_teams['Loc'] == 'H').astype(int)
all_teams['A'] = (all_teams['Loc'] == 'A').astype(int)
all_teams['N'] = (all_teams['Loc'] == 'N').astype(int)
all_teams.drop('Loc', axis=1, inplace=True)
combined_teams = all_teams.groupby(['Season', 'TeamID']).sum()
combined_teams['Game_played'] = combined_teams['win'] + combined_teams['loss']


display (combined_teams)


In [ ]:
reg_input = pd.DataFrame()
reg_input['win_ratio'] = combined_teams['win'] / combined_teams['Game_played']
reg_input['ppg'] = combined_teams['Score'] / combined_teams['Game_played']
reg_input['opp_ppg'] = combined_teams['oppScore'] / combined_teams['Game_played']
reg_input['pts_ratio'] = combined_teams['Score'] / combined_teams['oppScore']
reg_input['ot_per_game'] = combined_teams['NumOT'] / combined_teams['Game_played']

reg_input['FGM3pergame'] = combined_teams['FGM3'] / combined_teams['Game_played']
reg_input['FG3ratio'] = combined_teams['FGM3'] / combined_teams['FGA3']
reg_input['FG3_allowedpergame'] = combined_teams['oppFGM3'] / combined_teams['Game_played']

reg_input['FGMpergame'] = combined_teams['FGM'] / combined_teams['Game_played']
reg_input['FGratio'] = combined_teams['FGM'] / combined_teams['FGA']
reg_input['FG_allowedpergame'] = combined_teams['oppFGM'] / combined_teams['Game_played']

reg_input['FTMpergame'] = combined_teams['FTM'] / combined_teams['Game_played']
reg_input['FTratio'] = combined_teams['FTM'] / combined_teams['FTA']
reg_input['FT_allowedpergame'] = combined_teams['oppFTM'] / combined_teams['Game_played']

reg_input['ORratio'] = combined_teams['OR']/(combined_teams['OR']+combined_teams['oppDR'])
reg_input['DRratio'] = combined_teams['DR']/(combined_teams['DR']+combined_teams['oppOR'])

reg_input['Ast_per_g'] = combined_teams['Ast']/combined_teams['Game_played']
reg_input['To_per_g'] = combined_teams['TO']/combined_teams['Game_played']
reg_input['Stl_per_g'] = combined_teams['Stl']/combined_teams['Game_played']
reg_input['Blk_per_g'] = combined_teams['Blk']/combined_teams['Game_played']
reg_input['Pf_per_g'] = combined_teams['PF']/combined_teams['Game_played']

display(reg_input)
display(W_seeds_merge_WTeamConferences)
reg_input_tourteams_with_conf = pd.merge(left=reg_input,right=\
                                             W_seeds_merge_WTeamConferences,on=['Season','TeamID'],\
                                                  how='inner')
reg_input_tourteams_with_conf.set_index(['Season', 'TeamID'], inplace=True)


display(reg_input_tourteams_with_conf)
display(reg_input.describe())




In [ ]:
seeds_dict = seeds.set_index(['Season', 'TeamID'])
tour_comp = pd.read_csv(input_dir+"WNCAATourneyCompactResults.csv")
#display(tour_comp)
tour_input = pd.DataFrame()

winners = pd.DataFrame()
losers = pd.DataFrame()
winners[['Season', 'Team1', 'Team2']] = tour_comp[['Season', 'WTeamID', 'LTeamID']]
losers[['Season', 'Team1', 'Team2']] = tour_comp[['Season', 'LTeamID', 'WTeamID']]
winners['Result'] = 1
losers['Result'] = 0

tour_input = pd.concat([winners, losers])
tour_input = tour_input[tour_input['Season']>=2010].reset_index(drop = True)

t1_seed = []
t2_seed = []

for x in range(len(tour_input)) : 
    idx = (tour_input['Season'][x], tour_input['Team1'][x])
    seed = seeds_dict.loc[idx].values[0]
    if len(seed) == 4:
        seed = int(seed[1:-1])
    else:
        seed = int(seed[1:])
    t1_seed.append(seed)
    idx = (tour_input['Season'][x], tour_input['Team2'][x])
    seed = seeds_dict.loc[idx].values[0]
    if len(seed) == 4:
        seed = int(seed[1:-1])
    else:
        seed = int(seed[1:])
    t2_seed.append(seed)

tour_input['Team1_seed'] = t1_seed
tour_input['Team2_seed'] = t2_seed
display(tour_input)







In [ ]:
#can change
minus = []

for x in range(len(tour_input)):
    idx = (tour_input['Season'][x], tour_input['Team1'][x])
    team1_score = reg_input_tourteams_with_conf.loc[idx].copy()
    team1_score['Seed'] = tour_input['Team1_seed'][x]
    
    idx = (tour_input['Season'][x], tour_input['Team2'][x])
    team2_score = reg_input_tourteams_with_conf.loc[idx].copy()
    team2_score['Seed'] = tour_input['Team2_seed'][x]
    
    outscore = team1_score - team2_score
    outscore['Result'] = tour_input['Result'][x]
    minus.append(outscore)
    
minus = pd.DataFrame(minus)
display(minus)


In [ ]:
'''corrs = round(minus.corr(), 2)
pd.options.display.max_columns = None
pd.options.display.max_rows = None
#display(np.abs(corrs['Result']))

corrs = minus.corr()['Result'].abs().sort_values()
display(np.abs(corrs['Result']))
# Determine the threshold for the 15% least correlated columns
threshold = int(0.15 * len(corrs))


least_correlated_features = corrs[:threshold].index.tolist()

# Drop the 15% least correlated columns
features_to_keep = minus.columns.difference(least_correlated_features)
'''
pd.options.display.max_columns = 25
pd.options.display.max_rows = 25
corrs = minus.corr()['Result'].abs().sort_values()
display(corrs)  # This will display the sorted correlations.

# Determine the threshold for the 15% least correlated columns
threshold = int(0.15 * len(corrs))
least_correlated_features = corrs[:threshold].index.tolist()

# Now you can safely remove the least correlated features
features_to_keep = minus.columns.difference(least_correlated_features)

# Remove NaN features
minus = minus.dropna(axis=1, how='any')
features_to_keep = [feature for feature in features_to_keep if feature in minus.columns]
number_of_original_features = minus.shape[1] - 1

# Continue with your feature scaling and model training...




'''import seaborn as sns
import matplotlib.pyplot as plt


plt.figure(figsize=(15,10))
sns.heatmap(corrs)
plt.show()'''

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()


norm_minus = pd.DataFrame(scaler.fit_transform(minus), columns=minus.columns)
norm_minus_filtered = norm_minus[features_to_keep].dropna(axis=1)
#X_filtered = norm_minus_filtered[norm_minus_filtered.columns.difference(['Result'])].values

X = norm_minus_filtered.drop('Result', axis=1).values
y = norm_minus['Result'].values
display(minus)
display(norm_minus_filtered)

In [5]:
from tensorflow.keras.models import Sequential, Input, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression
import pandas as pd

# Assuming X, y and norm_minus_filtered are already defined as per your dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)
X_for_tf = norm_minus_filtered.drop('Result', axis=1)
y_for_tf = norm_minus_filtered['Result']
X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(X_for_tf, y_for_tf, test_size=0.2, random_state=9)

dtc = DecisionTreeRegressor(criterion='mse', max_depth=5, min_samples_split=10, random_state=9)
rfc = RandomForestRegressor(n_estimators=100, max_depth=5, min_samples_split=10, random_state=9)
svc = SVR(kernel='linear')
gnb = GaussianNB()
lr = LinearRegression()

# Neural network model
model = Sequential([
    Input(shape=(X_train_tf.shape[1],)),
    Dense(32, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='linear')  # Changed to linear for regression output
])

optimizer = Adam(learning_rate=0.0001)
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_absolute_error'])

early_stopping = EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=10,
    verbose=1,
    restore_best_weights=True
)

# Fit the regression models and make continuous predictions
dtc.fit(X_train, y_train)
y_pred_dtc_continuous = dtc.predict(X_test)
rfc.fit(X_train, y_train)
y_pred_rfc_continuous = rfc.predict(X_test)
svc.fit(X_train, y_train)
y_pred_svc_continuous = svc.predict(X_test)
gnb.fit(X_train, y_train)
y_pred_gnb_continuous = gnb.predict_proba(X_test)[:, 1]  # Use probability of the positive class
lr.fit(X_train, y_train)
y_pred_lr_continuous = lr.predict(X_test)

# Fit the neural network
model.fit(X_train_tf, y_train_tf, epochs=200, batch_size=32, validation_split=0.2, callbacks=[early_stopping])
y_pred_nn_continuous = model.predict(X_test_tf).flatten()

# Convert continuous predictions to binary outcomes
y_pred_dtc = (y_pred_dtc_continuous > 0.5).astype(int)
y_pred_rfc = (y_pred_rfc_continuous > 0.5).astype(int)
y_pred_svc = (y_pred_svc_continuous > 0.5).astype(int)
y_pred_gnb = (y_pred_gnb_continuous > 0.5).astype(int)
y_pred_lr = (y_pred_lr_continuous > 0.5).astype(int)
y_pred_nn = (y_pred_nn_continuous > 0.5).astype(int)

def calculate_metrics(y_test, y_pred):
    return {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1 Score': f1_score(y_test, y_pred),
        'ROC-AUC Score': roc_auc_score(y_test, y_pred)
    }

# Calculate metrics for each model
models_metrics = {
    'Linear Regression': calculate_metrics(y_test, y_pred_lr),
    'Decision Tree': calculate_metrics(y_test, y_pred_dtc),
    'Random Forest': calculate_metrics(y_test, y_pred_rfc),
    'Support Vector Regression': calculate_metrics(y_test, y_pred_svc),
    'Naive Bayes (Probabilities)': calculate_metrics(y_test, y_pred_gnb),
    'Neural Network': calculate_metrics(y_test, y_pred_nn)
}

# Create a DataFrame from the dictionary
df_metrics = pd.DataFrame(models_metrics).T  # Transpose to get models as rows

# Display the DataFrame
print(df_metrics)





'''from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=9)
X_for_tf = norm_minus_filtered.drop('Result', axis=1)
y_for_tf = norm_minus_filtered['Result']
X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(
    X_for_tf, y_for_tf, test_size=0.2, random_state=9)




dtc = DecisionTreeClassifier(criterion='gini', max_depth = 5, min_samples_split = 10, random_state=9)
rfc = RandomForestClassifier(n_estimators=100, max_depth = 5, min_samples_split = 10, random_state=9)  # You can adjust additional parameters here
svc = SVC(kernel='linear')  # You can change the kernel based on your dataset characteristics
gnb = GaussianNB()
lr = LogisticRegression()


model = Sequential([
    Input(shape=(X_train_tf.shape[1],)),  # Use Input layer to specify input shape
    Dense(32, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

optimizer = Adam(learning_rate=0.0001) 
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

early_stopping = EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=10,
    verbose=1,
    restore_best_weights=True)

dtc.fit(X_train, y_train)
y_pred_dtc = dtc.predict(X_test)
rfc.fit(X_train, y_train)
y_pred_rfc = rfc.predict(X_test)
svc.fit(X_train, y_train)
y_pred_svc = svc.predict(X_test)
gnb.fit(X_train, y_train)
y_pred_gnb = gnb.predict(X_test)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
model.fit(
    X_train_tf, 
    y_train_tf, 
    epochs=200, 
    batch_size=32, 
    validation_split=0.2,  
    callbacks=[early_stopping])
y_pred_nn = model.predict(X_test_tf)


def calculate_metrics(y_test, y_pred):
    return {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1 Score': f1_score(y_test, y_pred),
        'ROC-AUC Score': roc_auc_score(y_test, y_pred)
    }

# Calculate metrics for each model
models_metrics = {
    'Linear Regression': calculate_metrics(y_test, y_pred_lr),
    'Decision Tree': calculate_metrics(y_test, y_pred_dtc),
    'Random Forest': calculate_metrics(y_test, y_pred_rfc),
    'Support Vector Classifier': calculate_metrics(y_test, y_pred_svc),
    'Gaussian Naive Bayes': calculate_metrics(y_test, y_pred_gnb),
    # For the Neural Network, ensure y_pred_nn is properly processed if needed
    'Neural Network': calculate_metrics(y_test, y_pred_nn.round())
}

# Create a DataFrame from the dictionary
df_metrics = pd.DataFrame(models_metrics).T  # Transpose to get models as rows

# Display the DataFrame
print(df_metrics)
'''



/Users/lukuanyi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


ImportError: cannot import name 'Input' from 'tensorflow.keras.models' (/Users/lukuanyi/Library/Python/3.9/lib/python/site-packages/keras/api/_v2/keras/models/__init__.py)